In [1]:
from skimage.metrics import structural_similarity as ssim
from math import log10, sqrt
import numpy as np
import cv2
import argparse
import glob
import os
import re

from google.colab import drive
drive.mount("/content/drive")
project_dir = '/content/drive/MyDrive/Deep Learning Project Private/f1_models_m1_images_results/final/'

def options():
 parser = argparse.ArgumentParser(description="Read image metadata")
 parser.add_argument("-o", "--first", help="Input image file.", required=True)
 parser.add_argument("-c", "--second", help="Input image file.", required=True)
 args = parser.parse_args()
 return args

def mse(imageA, imageB):
 # the 'Mean Squared Error' between the two images is the sum of the squared difference between the two images
 mse_error = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
 mse_error /= float(imageA.shape[0] * imageA.shape[1])
	
 # return the MSE. The lower the error, the more "similar" the two images are.
 return mse_error

def compare(imageA, imageB):
 # Calculate the MSE and SSIM
 m = mse(imageA, imageB)
 s = ssim(imageA, imageB)

 # Return the SSIM. The higher the value, the more "similar" the two images are.
 return s

def calculate_SSIM(imageA, imageB): 
 # Convert the images to grayscale
 gray1 = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
 gray2 = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

 # Check for same size and ratio and report accordingly
 ho, wo, _ = imageA.shape
 hc, wc, _ = imageB.shape
 ratio_orig = ho/wo
 ratio_comp = hc/wc
 dim = (wc, hc)

 if round(ratio_orig, 2) != round(ratio_comp, 2):
  print("\nImages not of the same dimension. Check input.")
  exit()

 # Resize first image if the second image is smaller
 elif ho > hc and wo > wc:
  print("\nResizing original image for analysis...")
  gray1 = cv2.resize(gray1, dim)

 elif ho < hc and wo < wc:
  print("\nCompressed image has a larger dimension than the original. Check input.")
  exit()

 if round(ratio_orig, 2) == round(ratio_comp, 2):
  mse_value = mse(gray1, gray2)
  ssim_value = compare(gray1, gray2)
  # print("MSE:", mse_value)
  print("SSIM:", ssim_value)
  return ssim_value


  # PSNR
def PSNR(original, other):
    mse = np.mean((original - other) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    print(f"PSNR value is {psnr} dB")
    return psnr
  
if __name__ == "__main__":
  # get test IDs
  gt_files = glob.glob(project_dir + '/1*gt.png')

  test_ids = [int(os.path.basename(gt_file)[0:5]) for gt_file in gt_files]

  ssim_result = np.zeros(len(test_ids))
  psnr_result = np.zeros(len(test_ids))

  for k, gt_file in enumerate(gt_files):
    in_file = re.sub("_gt.", "_out.", gt_file)
    print(f"\nRunning: {gt_file.split('/')[-1]} and {in_file.split('/')[-1]}")

    gt_image = cv2.imread(gt_file)
    in_image = cv2.imread(in_file, 1)

    ssim_result[k] = calculate_SSIM(gt_image, in_image)
    psnr_result[k] = PSNR(gt_image, in_image)
  
  print(f"\nAVERAGE SSIM: {np.mean(ssim_result)}")
  print(f"AVERAGE PSNR: {np.mean(psnr_result)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Running: 10020_00_1_gt.png and 10020_00_1_out.png
SSIM: 0.032004508515614316
PSNR value is 27.754984621174444 dB

Running: 10020_00_10_gt.png and 10020_00_10_out.png
SSIM: 0.09777527723316023
PSNR value is 27.82705297447776 dB

Running: 10020_00_33_gt.png and 10020_00_33_out.png
SSIM: 0.2172720085507208
PSNR value is 27.99554632666407 dB

Running: 10019_00_10_gt.png and 10019_00_10_out.png
SSIM: 0.05552057632680971
PSNR value is 28.501832631441175 dB

Running: 10019_00_33_gt.png and 10019_00_33_out.png
SSIM: 0.12997837459325537
PSNR value is 28.545754974772443 dB

Running: 10019_00_3_gt.png and 10019_00_3_out.png
SSIM: 0.03003117066814342
PSNR value is 28.472819902780564 dB

Running: 10009_00_11_gt.png and 10009_00_11_out.png
SSIM: 0.14638644107424423
PSNR value is 27.91106950655914 dB

Running: 10009_00_66_gt.png and 10009_00_66_out.png
SSIM: 0.209821526607

ValueError: ignored